<a href="https://www.kaggle.com/code/atanurgl/mlbootcampproject?scriptVersionId=241770320" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


df = pd.read_csv('winequality-red.csv', sep=';')

# Kalite skorlarını 3 kategoriye bölüyorum
df['quality_category'] = pd.cut(
    df['quality'],
    bins=[0, 4, 6, 10],
    labels=['düşük', 'orta', 'yüksek']
)

# Kategorik etiketleri sayısala çeviriyorum
le = LabelEncoder()
df['quality_label'] = le.fit_transform(df['quality_category'])

print("Veri yüklendi ve işlendi.")
print(df[['quality', 'quality_category', 'quality_label']].sample(3))


Veri yüklendi ve işlendi.
      quality quality_category  quality_label
552         6             orta              1
14          5             orta              1
1317        6             orta              1


In [19]:
from sklearn.model_selection import train_test_split

# X ve Y özelliklerini ayırdım
X = df.drop(['quality', 'quality_category', 'quality_label'], axis=1)  # X için
y = df['quality_label']  # y için

# Eğitim ve test setlerini ayırıyorum
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Kontrol
print("\n 2. ADIM TAMAMLANDI: Veri bölündü!")
print("Eğitim Verisi Boyutu:", X_train.shape)
print("Test Verisi Boyutu:", X_test.shape)


 2. ADIM TAMAMLANDI: Veri bölündü!
Eğitim Verisi Boyutu: (1279, 11)
Test Verisi Boyutu: (320, 11)


In [20]:
#Dengesiz veri setimi dengelemek için SMOTE kullanıyorum
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)  

# Kontrol
print("\n 3. ADIM TAMAMLANDI: SMOTE uygulandı!")
print("Yeni Dağılım:")
print(pd.Series(y_res).value_counts())  


 3. ADIM TAMAMLANDI: SMOTE uygulandı!
Yeni Dağılım:
1    1057
0    1057
2    1057
Name: quality_label, dtype: int64


In [21]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    random_state=42,
    n_estimators=200  # Daha iyi ve hızlı output için
)
model.fit(X_res, y_res)

print("\n 4. ADIM TAMAMLANDI: Model eğitildi!")


 4. ADIM TAMAMLANDI: Model eğitildi!


In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print("\n SONUÇLAR:")
print(classification_report(y_test, y_pred, target_names=['düşük', 'orta', 'yüksek']))


📊 SONUÇLAR:
              precision    recall  f1-score   support

       düşük       0.29      0.36      0.32        11
        orta       0.94      0.88      0.91       262
      yüksek       0.64      0.81      0.72        47

    accuracy                           0.85       320
   macro avg       0.62      0.68      0.65       320
weighted avg       0.87      0.85      0.86       320



In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Hiperparametre kombinasyonları 
param_grid = {
    'n_estimators': [200, 300],  
    'max_depth': [15, 20, None],
    'min_samples_split': [2, 5]
}


grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,  
    scoring='f1_macro'
)

# X_res ve y_res ile eğittik
grid_search.fit(X_res, y_res)  

print(" En İyi Parametreler:", grid_search.best_params_)

 En İyi Parametreler: {'max_depth': 15, 'min_samples_split': 2, 'n_estimators': 200}


In [24]:
# parametrelerle modeli eğittim
best_model = grid_search.best_estimator_
best_model.fit(X_res, y_res)

print("\n 7. ADIM TAMAMLANDI: Final model eğitildi!")


 7. ADIM TAMAMLANDI: Final model eğitildi!


In [25]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred_best = best_model.predict(X_test)

print("\n FİNAL MODELİN TEST PERFORMANSI:")
print(classification_report(y_test, y_pred_best, target_names=['düşük', 'orta', 'yüksek']))

print("Karışıklık Matrisi:")
print(confusion_matrix(y_test, y_pred_best))



 FİNAL MODELİN TEST PERFORMANSI:
              precision    recall  f1-score   support

       düşük       0.25      0.36      0.30        11
        orta       0.95      0.87      0.91       262
      yüksek       0.65      0.87      0.75        47

    accuracy                           0.85       320
   macro avg       0.62      0.70      0.65       320
weighted avg       0.88      0.85      0.86       320

Karışıklık Matrisi:
[[  4   7   0]
 [ 12 228  22]
 [  0   6  41]]
